<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/stable/diffusers-inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion - Diffusers

Creating images with Stable Diffusion using the Diffusers library from Huggingface.  

[![Buy me a beer](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/buy-me-a-beer.png ) ](https://steinhaug.com/donate/)

In [1]:
#@title Check type of GPU and VRAM available, load notebook functions
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

import os

use_gdrive = True #@param {type:"boolean"}

if use_gdrive:
    if not os.path.isdir('/content/drive/MyDrive'):
        from google.colab import drive
        drive.mount('/content/drive')

Tesla T4, 15360 MiB, 15101 MiB
Mounted at /content/drive


In [83]:
#@markdown Load functions
import os
import zipfile
import numpy as np
from PIL import Image

_B=True
_A=False
def return__isValidDir(directory):
    if os.path.isdir(directory):return _B
    else:return _A

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

def decompress_zip(zip_file, dest_folder, flatten=False):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Extract all files into the destination folder
        if flatten:
            for file_info in zip_ref.infolist():
                with zip_ref.open(file_info) as file, open(os.path.join(dest_folder, os.path.basename(file_info.filename)), 'wb') as output_file:
                    output_file.write(file.read())
        else:
            zip_ref.extractall(dest_folder)


def maxx(image_ref, max_width=256):
    # Calculate the new height while maintaining the aspect ratio
    aspect_ratio = image_ref.width / image_ref.height
    new_height = int(max_width / aspect_ratio)

    # Resize the image
    image_ref.thumbnail((max_width, new_height))
    return image_ref

palette = np.asarray([
    [0, 0, 0],
    [120, 120, 120],
    [180, 120, 120],
    [6, 230, 230],
    [80, 50, 50],
    [4, 200, 3],
    [120, 120, 80],
    [140, 140, 140],
    [204, 5, 255],
    [230, 230, 230],
    [4, 250, 7],
    [224, 5, 255],
    [235, 255, 7],
    [150, 5, 61],
    [120, 120, 70],
    [8, 255, 51],
    [255, 6, 82],
    [143, 255, 140],
    [204, 255, 4],
    [255, 51, 7],
    [204, 70, 3],
    [0, 102, 200],
    [61, 230, 250],
    [255, 6, 51],
    [11, 102, 255],
    [255, 7, 71],
    [255, 9, 224],
    [9, 7, 230],
    [220, 220, 220],
    [255, 9, 92],
    [112, 9, 255],
    [8, 255, 214],
    [7, 255, 224],
    [255, 184, 6],
    [10, 255, 71],
    [255, 41, 10],
    [7, 255, 255],
    [224, 255, 8],
    [102, 8, 255],
    [255, 61, 6],
    [255, 194, 7],
    [255, 122, 8],
    [0, 255, 20],
    [255, 8, 41],
    [255, 5, 153],
    [6, 51, 255],
    [235, 12, 255],
    [160, 150, 20],
    [0, 163, 255],
    [140, 140, 140],
    [250, 10, 15],
    [20, 255, 0],
    [31, 255, 0],
    [255, 31, 0],
    [255, 224, 0],
    [153, 255, 0],
    [0, 0, 255],
    [255, 71, 0],
    [0, 235, 255],
    [0, 173, 255],
    [31, 0, 255],
    [11, 200, 200],
    [255, 82, 0],
    [0, 255, 245],
    [0, 61, 255],
    [0, 255, 112],
    [0, 255, 133],
    [255, 0, 0],
    [255, 163, 0],
    [255, 102, 0],
    [194, 255, 0],
    [0, 143, 255],
    [51, 255, 0],
    [0, 82, 255],
    [0, 255, 41],
    [0, 255, 173],
    [10, 0, 255],
    [173, 255, 0],
    [0, 255, 153],
    [255, 92, 0],
    [255, 0, 255],
    [255, 0, 245],
    [255, 0, 102],
    [255, 173, 0],
    [255, 0, 20],
    [255, 184, 184],
    [0, 31, 255],
    [0, 255, 61],
    [0, 71, 255],
    [255, 0, 204],
    [0, 255, 194],
    [0, 255, 82],
    [0, 10, 255],
    [0, 112, 255],
    [51, 0, 255],
    [0, 194, 255],
    [0, 122, 255],
    [0, 255, 163],
    [255, 153, 0],
    [0, 255, 10],
    [255, 112, 0],
    [143, 255, 0],
    [82, 0, 255],
    [163, 255, 0],
    [255, 235, 0],
    [8, 184, 170],
    [133, 0, 255],
    [0, 255, 92],
    [184, 0, 255],
    [255, 0, 31],
    [0, 184, 255],
    [0, 214, 255],
    [255, 0, 112],
    [92, 255, 0],
    [0, 224, 255],
    [112, 224, 255],
    [70, 184, 160],
    [163, 0, 255],
    [153, 0, 255],
    [71, 255, 0],
    [255, 0, 163],
    [255, 204, 0],
    [255, 0, 143],
    [0, 255, 235],
    [133, 255, 0],
    [255, 0, 235],
    [245, 0, 255],
    [255, 0, 122],
    [255, 245, 0],
    [10, 190, 212],
    [214, 255, 0],
    [0, 204, 255],
    [20, 0, 255],
    [255, 255, 0],
    [0, 153, 255],
    [0, 41, 255],
    [0, 255, 204],
    [41, 0, 255],
    [41, 255, 0],
    [173, 0, 255],
    [0, 245, 255],
    [71, 0, 255],
    [122, 0, 255],
    [0, 255, 184],
    [0, 92, 255],
    [184, 255, 0],
    [0, 133, 255],
    [255, 214, 0],
    [25, 194, 194],
    [102, 255, 0],
    [92, 0, 255],
])

In [ ]:
#@markdown Install dependencies
from IPython.display import display, clear_output

!pip install -q git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/diffusers

#!pip3 install Cython
#!pip freeze | grep torch
#!pip freeze | grep diffusers
#!pip show torch | grep Version

!wget --content-disposition https://t.co/9WZvWVGYqR -P /content
!pip install /content/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!pip install peft

clear_output(); print(f"Done (note you need to restart runtime if you get accellerate errors)")

In [29]:
# /content/drive/MyDrive/models/checkpoints/steinhaug/steinhaug.ckpt
# /content/drive/MyDrive/models/checkpoints/steinhaug/kimsteinhaug-2000.ckpt
# /content/drive/MyDrive/models/checkpoints/steinhaug/kimsteinhaug-3000.ckpt
CKPT_FILEPATH = '/content/drive/MyDrive/models/checkpoints/steinhaug/mxmulti-5000.ckpt'
cpu_or_cuda = 'cuda'

model_path = '/content/drive/MyDrive/models/SD/sd1.5'

In [30]:
#@markdown 2.0: Initialize and load model
sheduler_mode = "EulerDiscrete" # @param ["DDIM","UniPCMultistep","DEISMultistep","KDPM2Discrete","EulerDiscrete","PNDM","HeunDiscrete","EulerAncestralDiscrete","DDPM","KDPM2AncestralDiscrete","LMSDiscrete","DPMSolverSDE","DPMSolverMultistep","DPMSolverSinglestep"]
# pipe.scheduler.compatibles

from IPython.display import display, clear_output
import torch
from torch import autocast
from IPython.display import display

from diffusers import StableDiffusionPipeline
#pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to(cpu_or_cuda)
pipe = StableDiffusionPipeline.from_single_file(CKPT_FILEPATH, torch_dtype=torch.float16)
pipe.to(cpu_or_cuda)

if sheduler_mode == 'DDIM':
    from diffusers import DDIMScheduler
    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DDPM':
    from diffusers import DDPMScheduler
    pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'HeunDiscrete':
    from diffusers import HeunDiscreteScheduler
    pipe.scheduler = HeunDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'UniPCMultistep':
    from diffusers import UniPCMultistepScheduler
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'EulerDiscrete':
    from diffusers import EulerDiscreteScheduler
    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'EulerAncestralDiscrete':
    from diffusers import EulerAncestralDiscreteScheduler
    pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'KDPM2Discrete':
    from diffusers import KDPM2DiscreteScheduler
    pipe.scheduler = KDPM2DiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'KDPM2AncestralDiscrete':
    from diffusers import KDPM2AncestralDiscreteScheduler
    pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DPMSolverSDEScheduler':
    from diffusers import DPMSolverSDEScheduler
    pipe.scheduler = DPMSolverSDEScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'LMSDiscreteScheduler':
    from diffusers import LMSDiscreteScheduler
    pipe.scheduler = LMSDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DPMSolverMultistepScheduler':
    from diffusers import DPMSolverMultistepScheduler
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'PNDM':
    from diffusers import PNDMScheduler
    pipe.scheduler = PNDMScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DEISMultistep':
    from diffusers import DEISMultistepScheduler
    pipe.scheduler = DEISMultistepScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DPMSolverSinglestep':
    from diffusers import DPMSolverSinglestepScheduler
    pipe.scheduler = DPMSolverSinglestepScheduler.from_config(pipe.scheduler.config)

pipe.enable_xformers_memory_efficient_attention()
g_cuda = None

seed = 1
g_cuda = torch.Generator(device=cpu_or_cuda)
g_cuda.manual_seed(seed)
saved_file_count = 1;

clear_output();
print(f'.ckpt file location: {CKPT_FILEPATH}')

.ckpt file location: /content/drive/MyDrive/models/checkpoints/steinhaug/mxmulti-5000.ckpt


In [90]:
#@markdown 2.1: Optional, reselect the scheduler or/and set token
#sheduler_mode = "EulerDiscrete" # @param ["DDIM","UniPCMultistep","DEISMultistep","KDPM2Discrete","EulerDiscrete","PNDM","HeunDiscrete","EulerAncestralDiscrete","DDPM","KDPM2AncestralDiscrete","LMSDiscrete","DPMSolverSDE","DPMSolverMultistep","DPMSolverSinglestep"]
sheduler_mode = "EulerDiscrete" # @param ["EulerDiscrete","DEISMultistep","EulerAncestralDiscrete","DPMSolverMultistep"]

YOUR_TOKEN = "mxxstein" #@param {type:"string"}
TOKEN_GENDER = "person" #@param ["person", "man", "woman"]

if sheduler_mode == 'DDIM':
    from diffusers import DDIMScheduler
    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DDPM':
    from diffusers import DDPMScheduler
    pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'HeunDiscrete':
    from diffusers import HeunDiscreteScheduler
    pipe.scheduler = HeunDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'UniPCMultistep':
    from diffusers import UniPCMultistepScheduler
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'EulerDiscrete':
    from diffusers import EulerDiscreteScheduler
    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'EulerAncestralDiscrete':
    from diffusers import EulerAncestralDiscreteScheduler
    pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'KDPM2Discrete':
    from diffusers import KDPM2DiscreteScheduler
    pipe.scheduler = KDPM2DiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'KDPM2AncestralDiscrete':
    from diffusers import KDPM2AncestralDiscreteScheduler
    pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DPMSolverSDEScheduler':
    from diffusers import DPMSolverSDEScheduler
    pipe.scheduler = DPMSolverSDEScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'LMSDiscreteScheduler':
    from diffusers import LMSDiscreteScheduler
    pipe.scheduler = LMSDiscreteScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DPMSolverMultistepScheduler':
    from diffusers import DPMSolverMultistepScheduler
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'PNDM':
    from diffusers import PNDMScheduler
    pipe.scheduler = PNDMScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DEISMultistep':
    from diffusers import DEISMultistepScheduler
    pipe.scheduler = DEISMultistepScheduler.from_config(pipe.scheduler.config)
elif sheduler_mode == 'DPMSolverSinglestep':
    from diffusers import DPMSolverSinglestepScheduler
    pipe.scheduler = DPMSolverSinglestepScheduler.from_config(pipe.scheduler.config)


In [ ]:
#@markdown 3.0: Generate image
from slugify import slugify
prompt = "highest quality,award-winning,professional erotic photo of (sexy pale woman rancher with beautiful blonde hair),((pretty face)),seductive look,(wearing cowgirl outfit,daisy dukes),athletic body,huge breasts,large ass,candid,(breast worship:0.7),(detailed skin),[beauty marks],freckles,hyperrealistic photography,outdoors,standing in front of old western tavern,(storm clouds in the sky above),stormy weather,natural lighting,(crepuscular rays:0.6),photographed on a Canon EOS R6,50mm prime lens,F/1.4,(highly detailed),cinestill 800,8mm film grain,dynamic angle,full body,cowboy shot" #@param {type:"string"}
negative_prompt = "child,childlike,(monochrome:1.3),(oversaturated:1.3),bad hands,lowers,3d render,cartoon,long body,((blurry)),duplicate,((duplicate body parts)),[wrong|anatomy],(disfigured),(poorly drawn),(extra limbs),fused fingers,extra fingers,(twisted),malformed hands,((((mutated hands and fingers)))),contorted,conjoined,((missing limbs)),logo,signature,text,words,low res,boring,mutated,artifacts,bad art,gross,ugly,poor quality,low quality,kid,missing asshole," #@param {type:"string"}
token_name = YOUR_TOKEN + " " + TOKEN_GENDER + ", " + YOUR_TOKEN
num_samples = 1 #@ param {type:"number"}
guidance_scale = 8 #@ param {type:"number"}
num_inference_steps = 23 #@param {type:"number"}
width = "512" #@ param ["512", "768", "1280", "1536"] {allow-input: true}
height = "768" #@ param ["512", "768", "1280", "1536"] {allow-input: true}
width = int(width)
height = int(height)
custom_seed = 2616847730 #@param {type:"number"}
new_seed = custom_seed
save_images_path = "/content/drive/MyDrive/AI-Images-Manual"

#prompt = prompt.replace("__token__", token_name)

x = token_name.split(",")
for index, value in enumerate(x):
    if index == 0:
        prompt = prompt.replace("__token__", value)
    else:
        prompt = prompt.replace("__token" + str(index) + "__", value)

#raise Exception(1);

if num_samples > 1:
    print("You may need to scroll to see the rest of the images as they appear...")

if new_seed:
    g_cuda = torch.Generator(device='cuda')
    g_cuda.manual_seed(new_seed)

if len(save_images_path):
    tmp = save_images_path.split("/")
    if len(tmp) == 1:
        save_images_path = "/content/" + save_images_path
    from pathlib import Path
    path = Path(save_images_path)
    if not path.exists():
        print(f"[*] Create save directory...")
        path.mkdir(parents = False, exist_ok = False)
    try:
        if not image_save_count:
            print('Darn we need this one!')
    except NameError:
        image_save_count = 1

print(f"[*] Prompt used: {prompt}")

gen_images = []

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    gen_images.append(img.copy())
    display(img)
    if len(save_images_path):
        precount = f'{image_save_count:04d}'
        image_filename = slugify(precount + '_' + prompt.replace(" ", '_')[:240]) + '.png'
        img.save(save_images_path + "/" + image_filename)
        image_save_count += 1

In [ ]:
#@markdown 3.1: continue, generating with array of schedulers, from previous block

sheduler_list = [
    "EulerDiscrete","DDIM","UniPCMultistep","DEISMultistep","KDPM2Discrete",
    "PNDM","HeunDiscrete","EulerAncestralDiscrete","DDPM",
    "KDPM2AncestralDiscrete","LMSDiscrete",
    "DPMSolverSDE","DPMSolverMultistep","DPMSolverSinglestep"
]
sheduler_list = ["EulerDiscrete","DEISMultistep","EulerAncestralDiscrete","DPMSolverMultistep"]

from slugify import slugify
num_samples = 1 #@ param {type:"number"}
guidance_scale = 8 #@param {type:"number"}
num_inference_steps = 23 #@param {type:"number"}
width = "512" #@param ["512", "768", "1280", "1536"] {allow-input: true}
height = "768" #@param ["512", "768", "1280", "1536"] {allow-input: true}
width = int(width)
height = int(height)
custom_seed = 2616847730 #@param {type:"number"}
new_seed = custom_seed
save_images_path = "/content/drive/MyDrive/AI-Images-Manual"


if num_samples > 1:
    print("You may need to scroll to see the rest of the images as they appear...")

if len(save_images_path):
    tmp = save_images_path.split("/")
    if len(tmp) == 1:
        save_images_path = "/content/" + save_images_path
    from pathlib import Path
    path = Path(save_images_path)
    if not path.exists():
        print(f"[*] Create save directory...")
        path.mkdir(parents = False, exist_ok = False)
    try:
        if not image_save_count:
            print('Darn we need this one!')
    except NameError:
        image_save_count = 1

print(f"[*] Prompt used: {prompt}")

gen_images = []

for sheduler_mode in sheduler_list:
    if sheduler_mode == 'DDIM':
        from diffusers import DDIMScheduler
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'DDPM':
        from diffusers import DDPMScheduler
        pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'HeunDiscrete':
        from diffusers import HeunDiscreteScheduler
        pipe.scheduler = HeunDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'UniPCMultistep':
        from diffusers import UniPCMultistepScheduler
        pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'EulerDiscrete':
        from diffusers import EulerDiscreteScheduler
        pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'EulerAncestralDiscrete':
        from diffusers import EulerAncestralDiscreteScheduler
        pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'KDPM2Discrete':
        from diffusers import KDPM2DiscreteScheduler
        pipe.scheduler = KDPM2DiscreteScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'KDPM2AncestralDiscrete':
        from diffusers import KDPM2AncestralDiscreteScheduler
        pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'DPMSolverSDEScheduler':
        from diffusers import DPMSolverSDEScheduler
        pipe.scheduler = DPMSolverSDEScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'LMSDiscreteScheduler':
        from diffusers import LMSDiscreteScheduler
        pipe.scheduler = LMSDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'DPMSolverMultistepScheduler':
        from diffusers import DPMSolverMultistepScheduler
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'PNDM':
        from diffusers import PNDMScheduler
        pipe.scheduler = PNDMScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'DEISMultistep':
        from diffusers import DEISMultistepScheduler
        pipe.scheduler = DEISMultistepScheduler.from_config(pipe.scheduler.config)
    elif sheduler_mode == 'DPMSolverSinglestep':
        from diffusers import DPMSolverSinglestepScheduler
        pipe.scheduler = DPMSolverSinglestepScheduler.from_config(pipe.scheduler.config)

    if new_seed:
        g_cuda = torch.Generator(device=cpu_or_cuda)
        g_cuda.manual_seed(new_seed)

    print(" ")
    print(f"MODE: {sheduler_mode}")
    with autocast("cuda"), torch.inference_mode():
        images = pipe(
            prompt,
            height=height,
            width=width,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_samples,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=g_cuda
        ).images

    for img in images:
        gen_images.append(img.copy())
        display(img)
        if len(save_images_path):
            precount = f'{image_save_count:04d}'
            image_filename = slugify(precount + '_' + prompt.replace(" ", '_')[:240]) + '.png'
            img.save(save_images_path + "/" + image_filename)
            image_save_count += 1

In [ ]:
print("50 steps")
image_grid(gen_images, 2, 7)

In [ ]:
print("20 steps")
image_grid(gen_images, 2, 7)

In [ ]:
#@markdown **AI-Manual - All options mode**
from slugify import slugify
prompt = "mxxstein person" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
token_name = YOUR_TOKEN + " " + TOKEN_GENDER + ", " + YOUR_TOKEN
num_samples = 4 #@param {type:"number"}
guidance_scale = 7.5
num_inference_steps = 30 #@param {type:"number"}
width = "512"
height = "512" #@param ["512", "768", "1280", "1536"] {allow-input: true}
width = int(width)
height = int(height)
custom_seed = None
new_seed = custom_seed
save_images_path = "/content/drive/MyDrive/AI-Images-Manual"

#prompt = prompt.replace("__token__", token_name)

x = token_name.split(",")
for index, value in enumerate(x):
    if index == 0:
        prompt = prompt.replace("__token__", value)
    else:
        prompt = prompt.replace("__token" + str(index) + "__", value)

#raise Exception(1);

if num_samples > 1:
    print("You may need to scroll to see the rest of the images as they appear...")

if new_seed:
    g_cuda = torch.Generator(device='cuda')
    g_cuda.manual_seed(new_seed)

if len(save_images_path):
    tmp = save_images_path.split("/")
    if len(tmp) == 1:
        save_images_path = "/content/" + save_images_path
    from pathlib import Path
    path = Path(save_images_path)
    if not path.exists():
        print(f"[*] Create save directory...")
        path.mkdir(parents = False, exist_ok = False)
    try:
        if not image_save_count:
            print('Darn we need this one!')
    except NameError:
        image_save_count = 1

print(f"[*] Prompt used: {prompt}")

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)
    if len(save_images_path):
        precount = f'{image_save_count:04d}'
        image_filename = slugify(precount + '_' + prompt.replace(" ", '_')[:240]) + '.png'
        img.save(save_images_path + "/" + image_filename)
        image_save_count += 1